**.NET** extends C#’s support for `enums` through the `System.Enum` type. This type has two roles  
- Providing `type unification` for ***all enum types***
- Defining **static** `utility methods`

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p><strong>Type Unification:</strong></p>
<ul><li>هر نوع شمارشی (enum) که در C# تعریف می‌شود، به صورت ضمنی از <code>System.Enum</code> ارث‌بری می‌کند. این به معنای آن است که تمامی enumها نوعی از <code>System.Enum</code> هستند و بنابراین می‌توانند به عنوان <code>System.Enum</code> استفاده شوند.</li><li>این ویژگی به C# امکان می‌دهد تا عملیات‌ها و رفتارهای مشترکی را برای همه enumها فراهم کند، مثل تبدیل به رشته (string)، بررسی مقادیر معتبر و غیره.</li></ul>
</div>

In [ ]:
enum Nut { Walnut, Hazelnut, Macadamia }
enum Size { Small, Medium, Large }

Display (Nut.Macadamia); // Nut.Macadamia
Display (Size.Large); // Size.Large

void Display (Enum value)
{
    Console.WriteLine (value.GetType().Name + "." + value.ToString());
}

void DisplayValues(Enum type)
{
       Array values = Enum.GetValues(type.GetType());
        foreach (var value in values)
        {
            Console.WriteLine(value);
        }
}

### Enum Conversions

There are three ways to represent an enum value:
- As an `enum member`
- As its `underlying integral value`
- As a `string`

#### Enum to integral conversions 

In [ ]:
[Flags]
public enum BorderSides { Left=1, Right=2, Top=4, Bottom=8 }

int i = (int) BorderSides.Top; // i == 4
BorderSides side = (BorderSides) i; // side == BorderSides.Top

//You can cast a System.Enum instance to its integral type in the same way.
static int GetIntegralValue (Enum anyEnum)
{
    return (int) (object) anyEnum;
}
//the method we just wrote would crash
//if passed an enum whose integral type was long.

//thsi method works with an enum of any integral type
static decimal GetAnyIntegralValue (Enum anyEnum)
{
    return Convert.ToDecimal (anyEnum);
}
//This works because every integral type (including ulong) can be converted to
//decimal without loss of information.

//second approch is to call Enum.GetUnderlyingType then call Convert.ChangeType
static object GetBoxedIntegralValue (Enum anyEnum)
{
    Type integralType = Enum.GetUnderlyingType (anyEnum.GetType());
    return Convert.ChangeType (anyEnum, integralType);
}
//This preserves the original integral type, as the following example shows
object result = GetBoxedIntegralValue (BorderSides.Top);
Console.WriteLine (result); // 4
Console.WriteLine (result.GetType()); // System.Int32

//The third approach is to call Format or ToString specifying the "d" or "D" format string.
static string GetIntegralValueAsString (Enum anyEnum)
{
    return anyEnum.ToString ("D"); // returns something like "4"
}

####  Integral to enum conversions

`Enum.ToObject` converts an **integral value** to an **enum instance** of the `given type`

In [ ]:
[Flags]
public enum BorderSides { Left=1, Right=2, Top=4, Bottom=8 }

object bs = Enum.ToObject (typeof (BorderSides), 3);
Console.WriteLine (bs); // Left, Right

//This is the dynamic equivalent of the following
BorderSides bs1 = (BorderSides) 3;

//ToObject is overloaded to accept all integral types as well as object.



#### String conversions 

To ***convert an enum to a string***, you can either call the **static** `Enum.Format` method or call `ToString` on the instance.  
- `G` for default formatting behavior
- `D` to emit the underlying integral value as a string
- `X` for the same in hexadecimal
- `F` to format combined members of an enum without the Flags attribute.

In [ ]:
[Flags]
public enum BorderSides { Left=1, Right=2, Top=4, Bottom=8 }

BorderSides bs1 = BorderSides.Right;
BorderSides bs2 = BorderSides.Left | BorderSides.Right;

Console.WriteLine(bs1.ToString("G")); //Right
Console.WriteLine(bs2.ToString("G")); //Left, Right

Console.WriteLine(bs1.ToString("D")); //2
Console.WriteLine(bs2.ToString("D")); //3

Console.WriteLine(bs1.ToString("X")); //00000002
Console.WriteLine(bs2.ToString("X")); //00000003

Console.WriteLine(bs1.ToString("F")); //Right
Console.WriteLine(bs2.ToString("F")); //Left, Right

public enum BorderSidesWhitoutFlag { Left=1, Right=2, Top=4, Bottom=8 }
BorderSidesWhitoutFlag bsUnknown = (BorderSidesWhitoutFlag)10;
Console.WriteLine(bsUnknown.ToString("G")); //10
Console.WriteLine(bsUnknown.ToString("F")); //Right, Bottom

`Enum.Parse` converts a `string` to an `enum`. It accepts the enum type and a string that `can include multiple members`

In [ ]:
[Flags]
public enum BorderSides { Left=1, Right=2, Top=4, Bottom=8 }

BorderSides leftRight = 
    (BorderSides) Enum.Parse ( typeof (BorderSides), "Left, Right");

### Enumerating Enum Values

`Enum.GetValues` returns an array comprising `all members` of a particular `enum type`

In [ ]:
[Flags]
public enum BorderSides { Left=1, Right=2, Top=4, Bottom=8 }

foreach (Enum value in Enum.GetValues (typeof (BorderSides)))
 Console.WriteLine (value);

`Enum.GetNames` performs the same function but returns an `array of strings`.

### How Enums Work

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>در C#، Enum ها اساساً مجموعه‌ای از ثابت‌های عددی هستند که یک نام قابل فهم به آنها داده شده است. این نام‌ها برای خوانایی و قابلیت استفاده بیشتر کد مورد استفاده قرار می‌گیرند. اما در سطح پایین‌تر و در زمان اجرای برنامه (runtime)، Enum ها در واقع تفاوتی با مقادیر عددی خود ندارند. این یعنی یک مقدار Enum در CLR (Common Language Runtime) همان مقدار عددی پایه خود را دارد.</p>

<p>تعریف Enum در CLR (محیط اجرای زبان مشترک) به شکل یک زیرنوع از System.Enum است که دارای فیلدهای عددی ثابت برای هر عضو می‌باشد. این ساختار باعث می‌شود که استفاده از Enum ها بسیار کارآمد باشد و هزینه اجرایی آن‌ها برابر با استفاده از ثابت‌های عددی باشد.</p>

<strong>نکته‌ای درباره ایمنی نوع (Type Safety)</strong>

<p>به دلیل اینکه Enum ها اساساً مقادیر عددی هستند، نوع ایمنی قوی (Strong Type Safety) در زمان اجرا ندارند. به عبارت دیگر، شما می‌توانید بدون دریافت خطا، یک مقدار عددی به یک Enum اضافه کنید، حتی اگر این مقدار معنایی نداشته باشد (مثل اضافه کردن 1234 به <code>BorderSides.Left</code>).</p>

<strong>روش‌های فراخوانی متدهای مجازی</strong>

<p>در زمان اجرا، یک شیء Enum تفاوتی با مقدار عددی خود ندارد، اما وقتی که شما متدهای مجازی مثل <code>ToString</code> یا <code>GetType</code> را فراخوانی می‌کنید، C# این شیء Enum را قبل از فراخوانی این متدها جعبه‌بندی (Boxing) می‌کند. این جعبه‌بندی باعث می‌شود که شیء Enum به نوع اصلی خود برگردد و متدهای مربوط به نوع Enum اجرا شوند. بنابراین، زمانی که شما <code>BorderSides.Right.ToString()</code> را فراخوانی می‌کنید، نتیجه <code>Right</code> است نه <code>2</code>.</p>

</div>